<a href="https://colab.research.google.com/github/WeedCat23/AI_HW3/blob/main/AI_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import

In [8]:
import torch
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms

import torch.utils.data 
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.nn.functional as F

# from tqdm.notebook import tqdm
import numpy as np
import random
import time
import matplotlib
import matplotlib.pyplot as plt

Define transform (data augmentation included)

In [9]:
img_norm = [(0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)]
train_trans = transforms.Compose([
    # Data augmentation
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(0.4),
    transforms.ToTensor(),
    transforms.Normalize(*img_norm)                                               
])

test_trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(*img_norm)
])

Load dataset and split

In [10]:
# Download datasets
train_ds = datasets.CIFAR10(root="./CIFAR10", transform=train_trans, train=True, download=True)
test_ds = datasets.CIFAR10(root="./CIFAR10", transform=test_trans, train=False, download=True)
# split train and valid data with ratio 9 : 1
n_train = len(train_ds)
split = n_train // 10
indices = list(range(len(train_ds)))
random.shuffle(indices)
train_sampler = indices[split:]
valid_sampler = indices[:split]
# Dataloader
train_dl = DataLoader(train_ds, batch_size = 1024, sampler = train_sampler)
valid_dl = DataLoader(train_ds, batch_size = 1024, sampler = valid_sampler)
test_dl = DataLoader(test_ds, batch_size = 1024)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./CIFAR10/cifar-10-python.tar.gz to ./CIFAR10
Files already downloaded and verified


One-hot encode

In [11]:
classes = train_ds.classes
print(classes)
linear_in_channel = 32*32*3

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


Hyper parameters

In [12]:
learning_rate = 0.001
batch_size = 1024
epochs = 50


Define Models

In [47]:
class Linear(nn.Module):
    def __init__(self):
      super(Linear, self).__init__()
      self.classifier = nn.Sequential(
        nn.Linear(linear_in_channel, 1024),
        nn.ReLU(),
        nn.Linear(1024, 1024),
        nn.ReLU(),
        nn.Linear(1024,256),
        nn.ReLU(),
        nn.Linear(256, 10),
        nn.Softmax()
        )

    def forward(self, x):
        x = self.classifier(x)
        x = x.view(x.size(0), 10)
        return x

In [43]:
class Convolution(nn.Module):
    def __init__(self):
      super(Convolution, self).__init__()
      self.classifier = nn.Sequential(
          
        # Conv block 1
        # input_shape=(3,32,32)  
        nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),# (16,32,32) ((W-K+2P)/S)+1
        # nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),# (16,16,16)
        nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=2),# (32,18,18)
        # nn.BatchNorm2d(32),
        nn.ReLU(),

        nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=1),# (64,16,16) 
        # nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),# (64,8,8)
        nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),# (32,8,8)
        # nn.BatchNorm2d(32),
        nn.ReLU(),


        # fully connected
        nn.Dropout(0.5),
        nn.Flatten(),
        nn.Linear(32*8*8, 10)
        )

    def forward(self, x):
        x = self.classifier(x)
        x = x.view(x.size(0), 10)
        return x

In [48]:
from torchsummary import summary
model1 = Linear()
summary(model1,(1,3*32*32))
model2 = Convolution()
summary(model2,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [-1, 1, 1024]       3,146,752
              ReLU-2              [-1, 1, 1024]               0
            Linear-3              [-1, 1, 1024]       1,049,600
              ReLU-4              [-1, 1, 1024]               0
            Linear-5               [-1, 1, 256]         262,400
              ReLU-6               [-1, 1, 256]               0
            Linear-7                [-1, 1, 10]           2,570
           Softmax-8                [-1, 1, 10]               0
Total params: 4,461,322
Trainable params: 4,461,322
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.04
Params size (MB): 17.02
Estimated Total Size (MB): 17.07
----------------------------------------------------------------
------------------------------------

In [ ]:

optimizer_1 = torch.optim.Adam(model1.parameters(), lr=learning_rate)
optimizer_2 = torch.optim.Adam(model2.parameters(), lr=learning_rate)
loss_func = nn.CrossEntropyLoss()   # the target label is not one-hotted

Define train and evaluate function

In [49]:
def train_model(model, loss_func, optimizer, input_shape, epoch, num_epochs, train_loader):
    training_loss = []
    training_accuracy = []
    correct_train = 0
    total_train = 0
    
    for i, (images, labels) in enumerate(train_loader):
        
        train = torch.autograd.Variable(images.view(input_shape))
        labels = torch.autograd.Variable(labels)
            
        optimizer.zero_grad() # Clear gradients
        outputs = model(train)
        train_loss = loss_func(outputs, labels) # Calculate softmax and cross entropy loss

        train_loss.backward() # Compute gradients
        optimizer.step()   # Update weights
            
        # torch.max(input, dim, keepdim=False, out=None) -> (Tensor, LongTensor)
        predicted = torch.max(outputs.data, 1)[1] # Get predictions from the maximum value
        total_train += len(labels) # Total number of labels
        correct_train += (predicted == labels).float().sum() # Total correct predictions   
        
    train_accuracy = 100 * correct_train / float(total_train)
    training_accuracy.append(train_accuracy)
    training_loss.append(train_loss.data)
    print('Epoch: [{}/{}]'.format(epoch+1, num_epochs))
    print('> Loss: {} Acc: {:.6f}% '.format(train_loss.data, train_accuracy))

    return training_loss, training_accuracy


In [51]:
# validate and test function
def evaluate(model, loss_func, input_shape, val_loader, test_loader, testing = False):
    loss = []
    accuracy = []  

    if testing :
        correct_test = 0
        total_test = 0
        conf_matrix = np.zeros((10, 10))
        class_correct = [0]*10
        class_total   = [0]*10
        
        for images, labels in test_loader:
            test = torch.autograd.Variable(images.view(input_shape))
            outputs = model(test)
            test_loss = loss_func(outputs, labels)
            predicted = torch.max(outputs.data, 1)[1]

            total_test += len(labels)
            correct_test += (predicted == labels).float().sum()         
            
            correct_tensor = predicted.eq(labels.data.view_as(predicted)) # compare, return Tensor(True, True,....)
            correct = np.squeeze(correct_tensor.numpy()) # return (True, True,....)
        
            for i in range(labels.size(0)):
                label = labels.data[i]
                class_correct[label] += correct[i].item()
                class_total[label] += 1
        
                # Update confusion matrix
                conf_matrix[label][predicted.data[i]] += 1
        
        test_accuracy = 100 * correct_test / float(total_test)
        accuracy.append(test_accuracy)
        loss.append(test_loss.data)
        print(' Test result --> Loss: {} Acc: {:.6f}%'.format(test_loss.data, test_accuracy))
        
        print('Accuracy of each digits')
        for i in range(10):
            if class_total[i] > 0:
                print('Test Accuracy of {}: {}, [{}/{}]' .format(
                    i, 100 * class_correct[i] / class_total[i],
                    np.sum(class_correct[i]), np.sum(class_total[i]))
                     )
            else:
                print('Test Accuracy of {}: N/A (no training examples)' .format(classes[i]))

        return loss, accuracy, conf_matrix
    else:
        correct_val = 0
        total_val = 0
        for images, labels in val_loader:
            
            val = torch.autograd.Variable(images.view(input_shape))
            outputs = model(val)

            val_loss = loss_func(outputs, labels)
            predicted = torch.max(outputs.data, 1)[1]

            total_val += len(labels)

            correct_val += (predicted == labels).float().sum()
            
        val_accuracy = 100 * correct_val / float(total_val)
        accuracy.append(val_accuracy)
        loss.append(val_loss.data)
        print(' Val   --> Loss: {} Acc: {:.6f}%'.format(val_loss.data, val_accuracy))
        return loss, accuracy, _